# Final Project 

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from mpl_toolkits.basemap import Basemap
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'
# %matplotlib notebook

## Import and clean up the data

In [2]:
# Import data
# All data (hour resolution)
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

In [3]:
# Individual stations (minuite resolution)
DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

DE = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DiscoveryElementary_temperature.dat', 
                 header=2)

HL = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Helgesen_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

JM = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JohnMuir_temperature.dat', 
                 header=2)

KT = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Keating_temperature.dat', 
                 header=2)

US = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

# Coastline 
MAP = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DataCoast.dat', 
                 sep='\s+', names = ["Lon", "Lat"])

### Clean up the All Stations data

In [4]:
# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][0]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][0]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4

### Clean up the minute resolution data

In [5]:
def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

In [6]:
DataM = [DC, DE, HL, JB, JM, KT, US]
DataMLabels = ['Deep Cove', 'Discovery', 'Helgesen', 'James Bay', 'John Muir', 'Keating', 'UVicSci']

In [7]:
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

DC = DataMIns[0]
DE = DataMIns[1]
HL = DataMIns[2]
JB = DataMIns[3]
JM = DataMIns[4]
KT = DataMIns[5]
US = DataMIns[6]

## Plot the map

In [8]:
# Here's the map
StationLoc = list(AS)
Lon = np.array([float(list(AS)[i][0]) for i in range(1, np.shape(list(AS))[0])])
Lat = np.array([float(list(AS)[i][1]) for i in range(1, np.shape(list(AS))[0])])

def map(title='WHAT\'S THE THE TITLE YOU DONKEY?!?!?', size=(6, 6), land='#FFFFFF', water='#FFFFFF'):
    fig, ax = plt.subplots(1, 1, figsize=size)
    ax.fill(MAP['Lon'], MAP['Lat'], land, zorder=0, label='')
    ax.plot(MAP['Lon'], MAP['Lat'], linewidth=size[0]/12, color='k', zorder=1, label='')

    ax.xaxis.set_major_formatter(StrMethodFormatter(r'{x:.1f}$^\circ$W'))
    ax.yaxis.set_major_formatter(StrMethodFormatter(r'{x:.2f}$^\circ$N'))
    mp.rc('xtick', labelsize=size[0]*1.7) 
    mp.rc('ytick', labelsize=size[0]*1.7) 
    plt.xticks(rotation=10)
    
    # Old and depricated string format:
    # Map2.xaxis.set_major_formatter(FormatStrFormatter(r'%1.1f$^\circ$W'))
    # Map2.yaxis.set_major_formatter(FormatStrFormatter(r'%1.2f$^\circ$N'))

    ax.set_xlim(-123.8, -123.25)
    ax.set_ylim(48.325, 48.725)
    ax.grid(dashes=(1,1), color='#555555', linewidth=size[0]/6)
    ax.set_facecolor(water)

    maptitle = title
    ax.set_title("\n".join(wrap(maptitle, 50)), fontsize=size[0]*2.25)
    ax.set_xlabel(r'Longitude', fontsize=size[0]*2)
    ax.set_ylabel(r'Latitude', fontsize=size[0]*2)

In [9]:
# Basemap version MAP
# -------------------
# mapx = 8
# mapy = 8

# plt.figure('Map', figsize=(mapx, mapy))
# Map = Basemap(projection='gnom', lat_0 = 48.53, lon_0 = -123.50, 
#                 width=mapx/2 * 10**4, height=mapy/2 * 10**4, resolution='f')

# lon, lat = Map(Lon, Lat)

# Map.fillcontinents(color="#c6ebc6", lake_color='#DDEEFF')
# Map.drawmapboundary(fill_color="#DDEEFF")
# Map.drawcoastlines()
# Map.drawparallels(np.arange(48.3, 49, 0.05), labels=[0, 1, 1, 0])
# Map.drawmeridians(np.arange(-124.0, -123.3, 0.1), labels=[1, 0, 0, 1])

# maptitle = 'Location of weather temperature measurement stations in south-east Vancouver Island'
# plt.title("\n".join(wrap(maptitle, 80)))
# # plt.xlabel('Longitude')
# # plt.ylabel('Latitude')
# Map.plot(lon, lat, marker='o', color='#CC00CC', markersize=4, linewidth=0)
# plt.show()

In [10]:
# Plot the base stations
map2title = 'Location of weather temperature measurement stations in south-east Vancouver Island'
map(map2title, land='#CCEECC', water='#DDEEFF')
plt.scatter(Lon, Lat, marker='o', s=10, color='#CC00CC', zorder=2)
plt.show()

<Figure size 432x432 with 1 Axes>

## Plot the heatmap 

In [11]:
t_int = ['2017-01-01 00:00', '2017-02-01 23:00']
AS_Plot = AS.loc[t_int[0] : t_int[1]]

# Date ranges to cover 
t_range = pd.date_range(AS_Plot.index[0], AS_Plot.index[-1], freq='D')
# Spacing of xtick lables in the heatmap
spacing = int(np.shape(AS_Plot)[0]/len(t_range))

In [24]:
fig, ax = plt.subplots(figsize=(20, 4))

# Plot the heatmap
AS_HM = sns.heatmap(data=AS_Plot.transpose(), cmap='bwr', xticklabels=spacing)

# Change the xticks format 
xticks = AS_HM.get_xticks()
xformat = [i.strftime('%d-%m-%Y') for i in t_range]
plt.xticks(xticks, xformat, rotation=90)

# Change the yticks format 
yticks = AS_HM.get_yticks()
yformat = [[list(AS)[i][1], list(AS)[i][0]] for i in range(np.shape(AS)[1])]
# yformat = list(AS)
plt.yticks(yticks, yformat)

plt.ylabel(r'Location [$^\circ$ N, $^\circ$ W]')
plt.title('Heat Map from {0} to {1}'.format(t_int[0], t_int[1]))
sns.despine()
plt.show()

<Figure size 1440x288 with 2 Axes>

In [25]:
AS

,-123.4460,-123.5010,-123.3210,-123.3930,-123.4860,-123.3700,-123.4770,-123.4570,-123.3380,-123.6960,...,-123.3590,-123.3810,-123.4860,-123.6430,-123.4260,-123.4040,-123.3240,-123.3090,-123.3110,-123.5490
,48.5745,48.5376,48.4655,48.4608,48.4356,48.5273,48.4529,48.6804,48.4562,48.3891,...,48.4683,48.4744,48.4205,48.6568,48.4533,48.6529,48.4865,48.4623,48.4359,48.4572
Time,,,,,,,,,,,,,,,,,,,,,
2008-01-01 00:00:00.000000,3.25,1.43,4.21,4.62,3.45,4.44,2.72,3.97,4.68,5.12,...,4.29,4.10,3.01,NaN,4.00,4.81,4.58,NaN,4.87,4.07
2008-01-01 01:00:02.879995,3.06,1.38,4.28,4.77,3.46,4.21,2.64,4.15,4.61,4.95,...,4.41,4.30,2.87,NaN,4.15,4.60,4.44,NaN,4.85,4.05
2008-01-01 01:59:57.120005,1.90,1.18,4.12,4.70,3.46,4.07,2.64,4.18,4.62,5.11,...,4.24,4.23,2.97,NaN,4.06,4.65,4.47,NaN,4.91,4.02
2008-01-01 03:00:00.000000,1.69,0.98,4.20,4.77,3.65,3.67,2.81,4.27,4.71,5.33,...,4.36,4.30,3.10,NaN,4.12,4.75,4.53,NaN,4.97,4.06
2008-01-01 04:00:02.879995,2.09,0.93,4.61,4.99,3.90,3.55,3.11,4.29,4.93,5.56,...,4.58,4.53,3.37,NaN,4.25,4.97,4.68,NaN,5.17,4.27
2008-01-01 04:59:57.120005,2.06,1.37,4.64,5.21,4.00,4.48,3.22,4.37,5.02,5.50,...,4.78,4.74,3.59,NaN,4.62,5.12,4.77,NaN,5.40,4.39
2008-01-01 06:00:00.000000,1.97,1.18,4.86,5.14,4.09,4.57,3.37,4.14,5.25,5.63,...,4.74,4.70,3.82,NaN,4.97,5.05,4.86,NaN,5.59,4.61
2008-01-01 07:00:02.879995,1.40,0.72,4.73,5.12,4.13,4.10,3.27,3.92,5.19,5.72,...,4.74,4.67,3.68,NaN,4.65,5.11,4.97,NaN,5.74,4.50
